In [2]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

import pandas as pd
import xmltodict
import json

##### 공공데이터활용지원센터_보건복지부 코로나19 감염 현황
> 활용신청하기
https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043376

In [8]:
key='NqkriQhELlufFDf5SKdA3FmroggD3BKi3Ri9Kcbn%2BeVGUH9W%2FRPzrb%2Bm%2Bfln57RZtq2sTsjetWjYfrKaoS%2FejQ%3D%3D'
url = f'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey={key}&'
queryParams = urlencode({ quote_plus('pageNo') : 1, 
                          quote_plus('numOfRows') : 10,
                          quote_plus('startCreateDt') : '20210119',
                          quote_plus('endCreateDt') : '202101207'})
url2 = url + queryParams
response = urlopen(url2)
# print(type(response)) # HTTPSresponse 
results = response.read().decode("utf-8")
# print(type(results))   # str
results_to_json = xmltodict.parse(results)
data = json.loads(json.dumps(results_to_json))
print(type(data))   # dic
print(data)   

<class 'dict'>
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'accDefRate': '1.4573418044', 'accExamCnt': '5192119', 'accExamCompCnt': '5043978', 'careCnt': '12028', 'clearCnt': '60180', 'createDt': '2021-01-20 09:38:16.549', 'deathCnt': '1300', 'decideCnt': '73508', 'examCnt': '148141', 'resutlNegCnt': '4970470', 'seq': '392', 'stateDt': '20210120', 'stateTime': '00:00', 'updateDt': '2021-04-20 15:23:25.562'}, {'accDefRate': '1.4661941407', 'accExamCnt': '5140315', 'accExamCompCnt': '4985970', 'careCnt': '12353', 'clearCnt': '59468', 'createDt': '2021-01-19 09:39:49.588', 'deathCnt': '1283', 'decideCnt': '73104', 'examCnt': '154345', 'resutlNegCnt': '4912866', 'seq': '391', 'stateDt': '20210119', 'stateTime': '00:00', 'updateDt': '2021-04-20 15:23:34.538'}]}, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '2'}}}


위의 데이터를 분석해서 아래의 형태의 CSV 파일을 만들자
```
날짜,누적확진자,격리해제환자,치료중환자,사망자수
20210119,73104,59468,12353,1283
20210120,73508,60180,12028,1300

```

In [4]:
corona=data['response']['body']['items']['item']
#추가하고 싶은 리스트 생성
Date=[]
Cnt=[]
clear_cnt=[]
care_cnt=[]
death_cnt=[]
exam_cnt=[]     # examCnt   검사중
for i in corona:
    Date.append(i['stateDt'])  #'stateDt': '20200801'
    Cnt.append(i['decideCnt'])  # decideCnt': '14336'   누적확진자
    clear_cnt.append(i['clearCnt'])   # 13233           격리 해제환자
    care_cnt.append(i['careCnt'])     # 802             치료중 환자
    death_cnt.append(i['deathCnt'])    #301             사망자 수


In [5]:
df=pd.DataFrame([Date,Cnt,clear_cnt,care_cnt,death_cnt]).T
df.columns=['날짜','누적확진자','격리해제환자','치료중환자','사망자수'] 
df

,날짜,누적확진자,격리해제환자,치료중환자,사망자수
0,20210120,73508,60180,12028,1300
1,20210119,73104,59468,12353,1283


In [6]:
df=df.sort_values(by='날짜', ascending=True)
df

,날짜,누적확진자,격리해제환자,치료중환자,사망자수
1,20210119,73104,59468,12353,1283
0,20210120,73508,60180,12028,1300


In [7]:
df.set_index('날짜', inplace=True)
df

,누적확진자,격리해제환자,치료중환자,사망자수
날짜,,,,
20210119,73104,59468,12353,1283
20210120,73508,60180,12028,1300


In [ ]:
df.to_csv('sample.csv')

In [ ]:
# column 명칭 알기 쉽게 바꾸기
col_names = {
    'accDefRate': '누적 환진률',
    'accExamCnt': '누적 검사 수',
    'accExamCompCnt': '누적 검사 완료 수',
    'careCnt': '치료중 환자 수',
    'clearCnt': '격리해제 수',
    'createDt': '등록일시분초',
    'deathCnt': '사망자 수',
    'decideCnt': '확진자 수',
    'examCnt': '검사진행 수',
    'resutlNegCnt':'결과 음성 수',
    'seq': '게시글번호(감염현황 고유값)',
    'stateDt': '기준일',
    'stateTime': '기준시간',
    'updateDt': '수정일시분초',
}